In [35]:
import os
# TAVILY API KEY를 기입합니다.
# os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

os.environ["LANGCHAIN_PROJECT"] = "AGENT TUTORIAL"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [36]:
from dotenv import load_dotenv

load_dotenv()

True

In [37]:
# TavilySearchReasults 클래스를 langchain_community.tools.tavily_search 모듈에서 가져옵니다.
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다.
# K=5 검색결과를 5개까지 가져오겠다는 의미.

search = TavilySearchResults(k=3)

In [10]:
# tools 리스트에 search와 retriever_tool을 추가
tools = [search]

In [2]:
from langchain_openai import ChatOpenAI 
from langchain.agents.format_scratchpad import format_log_to_messages
from langchain.agents.output_parsers import (ReActJsonSingleInputOutputParser,)
from langchain_openai import ChatOpenAI

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.pydantic_v1 import BaseModel, Field

from langchain.tools.render import render_text_description_and_args
from langchain_core.messages import AIMessage, HumanMessage

from langchain.tools import tool

local_llama3 = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    model="congnitivecomputations/dolphin-2.9-llama3-8b-gguf",
    temperature=0,
)

In [ ]:
@tool
def search_phonenumber(query: str) -> str:
    """장소에 대한 전화번호 검색 결과를 반환할 때 사용되는 도구"""
    return "판교 몽중헌 전화번호: 010-1234-5678\n\n서울 OOO 전화번호: 02-123-4567"

tools = [search_phonenumber]
llm_with_tools = local_llama3.bind_tools(tools)

다음으로, 에이전트를 안내하기 위해 사용하고 싶은 프롬프트를 선택
이 프롬프트의 내용을 보고 싶고 LangSmith에 접근할 수 있다면, 다음으로 이동
다운로드 받아 활용할 프롬프트: https://smith.lanchain.com/hub/hwchase17/openai-functions-agent

hub.pull()함수를 사용하여 리소스를 가져온 후,
prompt.messages를 통해 해당 리소스의 메시지에 접근.

In [68]:
from langchain import hub 

In [74]:
# hub에서 prompt를 가져온다.
json_prompt = hub.pull("teddynote/react-chat-json-korean")

In [ ]:
json_prompt

이제 LLM, 프롬프트 및 도구로 에이전트를 초기화할 수 있습니다. 에이전트는 입력을 받아 어떤 action을 취할지 결정하는 역할을 합니다.
중요한 것은 에이전트가 action들을 실행하지 않는 다는 것입니다. (agentexcutor에 의해 실행)

In [75]:
from langchain.agents import create_json_chat_agent
# OpenAI 함수 기반 에이전트를 생성.
# llm, tools, prompt를 인자로 사용.
agent = create_json_chat_agent(llama3, tools, json_prompt)

In [76]:
from langchain.agents import AgentExecutor
# AgentExecutor 클래스를 사용하여 agnet와 tools를 설정하고,
# 상세한 로그를 출력하도록 verbose를 true로 설정.
agent_executor = AgentExecutor(agent=agent, tools=tools, vector=True)

In [77]:
response = agent_executor.invoke(
    {"input": "2024년 19-20세 청년 동향에 대해 알려주세요."}
)
response["output"]

'2024년 19-20세 청년 동향은 고용률이 68.7%로 전년동월대비 0.9%p 상승, 실업률은 3.7%로 전년동월대비 0.1%p 상승, 실업자는 1,072천명으로 전년동월대비 48천명 (4.7%) 증가하는 등 다양한 추이들을 보였다. 또한, 2024년 청년지원정책 중에는 재학단계부터 맞춤형 지원 및 일경험 확대, 신규청년 국가기술자격 응시료 지원 등이 있다.'

In [73]:
response = agent_executor.invoke(
    {"input": "19-20세 청년 동향에 대해 PDF에서 알려주세요."}
)
print(f'답변: {response["output"]}')

답변: According to the provided PDF, the youth policy white paper 2021 reports that 19-20 year old youths are facing difficulties in education and employment due to the COVID-19 pandemic. The paper highlights that the youth unemployment rate has increased, and many youths are delaying their education due to financial difficulties. Additionally, the paper notes that the youth suicide rate has also increased, with economic difficulties, job problems, and feelings of loneliness and isolation being major contributing factors.
